In [43]:
import pandas as pd
import numpy as np
import haversine as hs
import folium 

In [44]:
cust_loc=pd.read_excel('customer_location.xlsx')
hotel_loc=pd.read_excel("Hotel_location.xlsx")

In [45]:
# concatenating lat and long to create a consolidated location as accepted by havesine function
cust_loc['coor'] = list(zip(cust_loc.Lat, cust_loc.Lon))
hotel_loc['coor'] = list(zip(hotel_loc.Lat, hotel_loc.Lon))

In [46]:
hotel_loc.head()

,Hotel,Lat,Lon,coor
0,A,28.500784,76.992369,"(28.500784, 76.992369)"
1,B,28.394231,77.050308,"(28.394231, 77.050308)"
2,C,28.426846,77.088834,"(28.426846, 77.088834)"
3,D,28.452686,77.093897,"(28.452686, 77.093897)"
4,E,28.554460,77.068127,"(28.55446, 77.068127)"


In [47]:
# defining a  function to calculate distance between two locations 
# loc1= location of an existing hotel
# loc2= location of customers

def distance_from(loc1,loc2): 
    dist=hs.haversine(loc1,loc2)
    return round(dist,2)


In [48]:
# running a loop which will parse customers location one by one to distance from function 
for _,row in hotel_loc.iterrows():
    cust_loc[row.Hotel]=cust_loc['coor'].apply(lambda x: distance_from(row.coor,x))

In [49]:
cust_loc.head()

,Location,Lat,Lon,coor,A,B,C,D,E,F,G,H
0,"Tirath Ram Hospitals Private Limited, Basai Ro...",28.457057,77.020177,"(28.4570571, 77.020177)",5.57,7.58,7.51,7.22,11.80,6.96,3.35,8.30
1,"G Block, Sector 82, Gurugram, Haryana 122004, ...",28.393189,76.959076,"(28.3931888, 76.9590762)",12.40,8.92,13.23,14.75,20.86,3.10,6.20,10.75
2,"GoMechanic - Bharat Motor Parts, Sector 17/18 ...",28.488146,77.057352,"(28.488146, 77.057352)",6.50,10.47,7.48,5.32,7.45,11.95,8.17,10.31
3,"E-303, Mapsko Paradise, Sector 83, Gurugram, H...",28.398214,76.962738,"(28.3982136, 76.9627379)",11.77,8.58,12.74,14.18,20.20,2.53,5.57,10.39
4,"249F, Phase IV, Udyog Vihar, Sector 18, Gurugr...",28.497367,77.082806,"(28.4973672, 77.0828057)",8.85,11.90,7.86,5.09,6.51,14.55,10.57,11.36


# Mapping through Folium 

In [50]:
def create_map(cust,hotel):
    m = folium.Map(location=[cust.Lat.mean(), cust.Lon.mean()], zoom_start=12, tiles='OpenStreetMap')
    
   
    
    for _, row in cust.iterrows():
         
        if row['Condition']=='Out':
            cluster_colour='blue'
        else:
            cluster_colour = 'green'
        
        folium.CircleMarker(
            location= [row['Lat'],row['Lon']],
            radius=5,
            popup= row[['Condition','Location']],
            color=cluster_colour,
            fill=True,
            fill_color=cluster_colour
            ).add_to(m)
    
    for _, row in hotel.iterrows(): 
        folium.Marker(
            location= [row['Lat'],row['Lon']],
            radius=5,
            popup= row['Hotel'],
            color='red',
            fill=True,
            fill_color='red'
             ).add_to(m)
    for _, row in hotel.iterrows(): 
        folium.Circle(
            location= [row['Lat'],row['Lon']],
            radius=3000,
            popup= row['Hotel'],
            color='black',
            fill=False,
        ).add_to(m)
        
    return m

In [51]:
hotel_list=['A','B','C','D','E','F','G','H']

In [52]:
cust_loc['Condition']='Out'

for i in hotel_list:
    cust_loc['Condition'][cust_loc.Condition=='Out']=cust_loc[cust_loc.Condition=='Out'][i].apply(lambda x: 'In' if x<3 else 'Out')
    

C:\Users\lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [53]:
m= create_map(cust_loc,hotel_loc)
m